<a href="https://colab.research.google.com/github/Aaradhy23/projects/blob/main/find_your_movie_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets sentence-transformers langchain-community chromadb --quiet #sentence transformers for open source application

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aaradhy024
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies


100%|██████████| 1.17M/1.17M [00:01<00:00, 1.18MB/s]

In [3]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings


In [5]:
data_df= pd.read_csv("/content/netflix-tv-shows-and-movies/NetFlix.csv")[["title","description"]]
data_df.head()

,title,description
0,3%,In a future where the elite inhabit an island ...
1,1920,An architect and his wife move into a castle t...
2,3 Heroines,Three Indonesian women break records by becomi...
3,Blue Mountain State: The Rise of Thadland,New NFL star Thad buys his old teammates' belo...
4,Blue Planet II,This sequel to the award-winning nature series...


In [6]:
#clean data
print(data_df.shape)
print()
print(data_df.isna().sum())

(7787, 2)

title          0
description    0
dtype: int64


In [7]:
Embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':'cuda'})

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
loader= DataFrameLoader(data_df,page_content_column="description")#here page content column is decided according to our database
descriptions= loader.load()
#description object creation

In [10]:
CHROMA_PATH= "chroma_movies_db"
chroma_db= Chroma.from_documents(
    descriptions,Embeddings, persist_directory=CHROMA_PATH
)

In [11]:
chroma_db#here we dont need to divid into chunks as we directly have the answer in dataset

In [12]:
del chroma_db

In [13]:
chroma_db=Chroma(persist_directory="/content/chroma_movies_db",embedding_function=Embeddings)
chroma_db

In [32]:
query="""
fault in our stars
"""

relevant_docs= chroma_db.similarity_search(query,k=5)#k determines closest entries

for doc in relevant_docs:
  print(doc.metadata["title"])
  print(doc.page_content)
  print()

The Wandering Earth
A looming collision with Jupiter threatens Earth as humans search for a new star. The planet's fate now lies in the hands of a few unexpected heroes.

The Universe: Ancient Mysteries Solved
From astronomical events to shapes and patterns in space, strange and wonderful phenomena are explored with modern science in this series.

Into the Night
Passengers and crew aboard a hijacked overnight flight scramble to outrace the sun as a mysterious cosmic event wreaks havoc on the world below.

Pinkfong & Baby Shark's Space Adventure
Joined by new friends from other planets, Pinkfong and Baby Shark explore outer space and search for missing star pieces to return home.

On the Real
A group of reality stars transitions into their newly achieved fame while dealing with the demands of an obnoxious TV executive and their regular lives.

